In [1]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import mapper, sessionmaker
from sqlalchemy import Column, Integer
from sqlalchemy.ext.declarative import declarative_base
from scrapy.selector import Selector
from sqlalchemy.sql import text as sa_text
import re

engine = create_engine('mysql+pymysql://root:@localhost/oerintegrationdb', echo=False)
Base = declarative_base(engine)


class Triple(Base):
    __tablename__ = 'triple'
    __table_args__ = {'autoload': True}


class CleanTriple(Base):
    __tablename__ = 'cleantriple'
    __table_args__ = {'autoload': True}

metadata = Base.metadata
Session = sessionmaker(bind=engine)
session = Session()

print("Ready DB")

In [2]:
triples = session.query(Triple).filter(Triple.subject=="Open textBooks")
engine.execute(sa_text('''TRUNCATE TABLE cleantriple''').execution_options(autocommit=True))

print("Cleaned")

Cleaned


# Get all information of courses

In [3]:
source = "Open textBooks"
session.add(CleanTriple(subject=source, predicate="hasName", object=source))

for triple in triples:
    # print(triple.subject)
    body = triple.object
    # Setting name
    name = Selector(text=body).xpath('//div[@class="twothird"]/h1/text()').get()
    session.add(CleanTriple(subject=source, predicate="hasBook", object=name))
    session.add(CleanTriple(subject=name, predicate="hasName", object=name))
    # print("name ", name)
    # Setting about
    about = Selector(text=body).xpath('//div[@id="AboutBook"]/p/text()').get()
    if about is not None and len(about)>1:
        session.add(CleanTriple(subject=name, predicate="hasAbout", object=about))
    
    pub_date = Selector(text=body).xpath('//div[@class="twothird"]/p[starts-with(text(),"Pub Date")]/text()').get()
    if pub_date is not None and len(pub_date)>1:
        pub_date= pub_date[pub_date.find(":")+1:]
        session.add(CleanTriple(subject=name, predicate="hasPubDateYear", object=pub_date)) 
    contents = Selector(text=body).xpath('//div[@id="TOC"]/ul/li/text()').getall()
    cont = 0
    for content in contents:
        cont+=1
        new_name = name +"-" +str(cont)
        session.add(CleanTriple(subject=name, predicate="hasChapter", object=new_name))
        # print(new_name)
        if content is not None and len(content)>1:
            session.add(CleanTriple(subject=new_name, predicate="chapterName", object=content))
            session.add(CleanTriple(subject=new_name, predicate="chapterOrder", object=cont))
    
    authors = Selector(text=body).xpath('//div[@id="AboutContributors"]/p/strong/text()').getall()
    for author in authors:
        # print(author)
        if author is not None and len(author)>1:
            session.add(CleanTriple(subject=name, predicate="hasAuthor", object=author))
    
    language = Selector(text=body).xpath('//div[@class="twothird"]/p[starts-with(text(),"Language")]/text()').get()
    if language is not None and len(language)>1:
        language = language[language.find(":")+1:]
        # print(language)
        session.add(CleanTriple(subject=name, predicate="hasLanguage", object=language)) 
    
    link = Selector(text=body).xpath('//ul[@class="BookTypes"]/li/a/@href').get()
    if link is not None and len(link)>1:
        # print(link)
        session.add(CleanTriple(subject=name, predicate="hasLink", object=link)) 
    
    stars = Selector(text=body).xpath('count(//div[@class="twothird"]/p[2]/img[@title="whole star"])').get()
    if stars is not None and len(stars)>1:
        # print(stars)
        session.add(CleanTriple(subject=name, predicate="hasStars", object=stars)) 
        
    isbn = Selector(text=body).xpath('//div[@class="twothird"]/p[starts-with(text(),"ISBN")]/text()').get()
    if isbn is not None and len(isbn)>1:
        isbn = isbn[isbn.find(":")+1:]
        # print(isbn)
        session.add(CleanTriple(subject=name, predicate="hasISBN", object=isbn)) 
    
    attribution = Selector(text=body).xpath('//p[@class="Badge-Condition"]/a/text()').get()
    if attribution is not None and len(attribution)>1:
        # print(attribution)
        session.add(CleanTriple(subject=name, predicate="hasAttribution", object=attribution))
        
    license = Selector(text=body).xpath('//p[@class="Badge-Condition"]/text()').getall()
    # print(license)
    if license is not None and len(license)==3:
        license = re.sub('\n+', '', license[2])
        license = license[license.index("C"):]
        # print(license)
        session.add(CleanTriple(subject=name, predicate="hasLicense", object=license))
    
    
    reviews = Selector(text=body).xpath('//div[starts-with(@id,"read")]').getall()
    
    review_cont = 0

    for review in reviews:
        review_cont += 1
        new_name = name +"-review-" +str(review_cont)
        session.add(CleanTriple(subject=name, predicate="hasReview", object=new_name))
        # print(review)
        review_meta = Selector(text=review).xpath('//div[starts-with(@id,"read")]/p[2]/reviewer/text()').get()
        # print(review_meta)
        name = review_meta[review_meta.find("y")+2:review_meta.find(",")]
        # print(name)
        date = review_meta[review_meta.find(" on ")+4:]
        # print(date)
        review_text = Selector(text=review).xpath('//div[starts-with(@id,"read")]/p[3]/text()').get()
        # print(review_text) 
        stars = Selector(text=review).xpath('count(//div[starts-with(@id,"read")]/p[1]/img[@title="whole star"])').get()
        # print(stars)
        session.add(CleanTriple(subject=new_name, predicate="hasName", object=name))
        session.add(CleanTriple(subject=new_name, predicate="hasDate", object=date))
        session.add(CleanTriple(subject=new_name, predicate="hasReview", object=review_text))
        session.add(CleanTriple(subject=new_name, predicate="hasStars", object=stars))
         
    image = Selector(text=body).xpath('//div[@class="third"]/img/@src').get()
    # print(license)
    if image is not None and len(image)>0: 
        image =  "https://open.umn.edu" +image
        session.add(CleanTriple(subject=name, predicate="hasImage", object=image))

print("Done")     
session.commit()



Done
